In [1]:
import os
from glob import glob
import pandas as pd

In [2]:
lbl_index = {
    'positive': 'positive',
    'neutral': 'neutral',
    'negative': 'negative'
}

dataset = {
    'positive': pd.DataFrame(columns=['positive_tweets']),
    'neutral': pd.DataFrame(columns=['neutral_tweets']),
    'negative': pd.DataFrame(columns=['negative_tweets'])
}

**Process Data**

In [3]:
# https://www.kaggle.com/shashank1558/preprocessed-twitter-tweets
path1 = '../datasets/preprocessed-twitter-tweets'

cache = pd.read_csv('{}\\processedPositive.csv'.format(path1)).transpose().reset_index()
cache.columns = ['positive_tweets']
dataset['positive'] = dataset['positive'].append(cache, ignore_index=True, sort=False)

cache = pd.read_csv('{}\\processedNeutral.csv'.format(path1)).transpose().reset_index()
cache.columns = ['neutral_tweets']
dataset['neutral'] = dataset['neutral'].append(cache, ignore_index=True, sort=False)

cache = pd.read_csv('{}\\processedNegative.csv'.format(path1)).transpose().reset_index()
cache.columns = ['negative_tweets']
dataset['negative'] = dataset['negative'].append(cache, ignore_index=True, sort=False)

In [4]:
# https://www.kaggle.com/c/twitter-sentiment-analysis2/overview
lbl_map = {
    'negative': 0,
    'positive': 1
}
path2 = '../datasets/twitter-sentiment-analysis2'

cache = pd.read_csv('{}/train.csv'.format(path2), encoding = "ISO-8859-1").drop(columns=['ItemID'])
dataset['positive'] = dataset['positive']['positive_tweets'].append(cache[cache.Sentiment==lbl_map['positive']].SentimentText, ignore_index=True).to_frame()
dataset['negative'] = dataset['negative']['negative_tweets'].append(cache[cache.Sentiment==lbl_map['negative']].SentimentText, ignore_index=True).to_frame()

In [5]:
# https://data.world/crowdflower/weather-sentiment
lbl_map = {
    'negative': 'Positive',
    'positive': 'Negative',
    'neutral': 'Neutral / author is just sharing information'
}
path3 = '../datasets'

cache = pd.read_csv('{}/weather-agg-DFE.csv'.format(path3))[['what_emotion_does_the_author_express_specifically_about_the_weather', 'tweet_text']]
cache = cache.rename(columns={'what_emotion_does_the_author_express_specifically_about_the_weather': 'Sentiment'})
cache = cache[~cache['Sentiment'].isin(['Tweet not related to weather condition', "I can't tell"])]
dataset['positive'] = dataset['positive'][0].append(cache[cache.Sentiment==lbl_map['positive']].tweet_text, ignore_index=True).to_frame()
dataset['negative'] = dataset['negative'][0].append(cache[cache.Sentiment==lbl_map['negative']].tweet_text, ignore_index=True).to_frame()
dataset['neutral'] = dataset['neutral']['neutral_tweets'].append(cache[cache.Sentiment==lbl_map['neutral']].tweet_text, ignore_index=True).to_frame()

In [6]:
for k in dataset.keys():
    print(k, dataset[k].shape)

positive (57914, 1)
neutral (1831, 1)
negative (44880, 1)


**Export Data**

In [7]:
for k in dataset.keys():
    dataset[k].to_csv('{}Tweets.csv'.format(k))
    dataset[k].columns = ['Tweets']
    dataset[k]['Labels'] = [lbl_index[k]]*dataset[k].shape[0]

pd.concat([dataset[k] for k in dataset.keys()]).to_csv('Tweets.csv')